#sample brocker with


""" mosquitto_sub -h 127.0.0.1 -p 1883 -t test-topic -v """

test
"""mosquitto_pub -h 127.0.0.1 -t test-topic -m '{"ping":"pong"}'"""

In [ ]:
#python3 -m pip install --user "paho-mqtt>=2.0"

In [1]:
from utils_deepstream import parse_ds_payload, fmt_num

def on_message(client, userdata, msg):
    objs = parse_ds_payload(msg.payload)
    if objs:
        for o in objs:
            x1, y1, x2, y2 = o["bbox"]
            cls = o.get("cls")
            conf = o.get("conf")
            parts = [
                o["last_seen"],
                msg.topic,
                o.get("sensor", ""),
                f"id={o['track_id']}",
                f"bbox=[{fmt_num(x1)},{fmt_num(y1)},{fmt_num(x2)},{fmt_num(y2)}]",
            ]
            if cls:
                parts.append(f"cls={cls}")
            if conf is not None:
                parts.append(f"conf={fmt_num(conf)}")
            line = " | ".join(p for p in parts if p)
            print(line)
        return


In [2]:
#!/usr/bin/env python3
from config import BrokerConfig as cfg
from utils_brocker_mqtt import MqttSubscriber


client = MqttSubscriber(cfg(), on_message=on_message).connect()
try:
    client.start_forever()
except KeyboardInterrupt:
    print("\n[MQTT] Stopped by user.")
finally:
    client.stop()

[MQTT] Connecting to 127.0.0.1:1883 … (topics=['test-topic'])
[MQTT] on_connect: Connected
[MQTT] Subscribed to 'test-topic' (qos=0)
2025-09-08T19:57:23.154Z | test-topic | sensor-1 | id=0 | bbox=[311.585,725.050,455.210,981.935] | cls=Person | conf=0.437
2025-09-08T19:57:23.195Z | test-topic | sensor-2 | id=3 | bbox=[1725.140,275.667,1778.130,410.314] | cls=Person | conf=0.356
2025-09-08T19:57:24.355Z | test-topic | sensor-4 | id=4 | bbox=[1279.860,263.861,1454.820,512.427] | cls=Person | conf=0.672
2025-09-08T19:57:24.395Z | test-topic | sensor-2 | id=3 | bbox=[1729.740,275.055,1782.080,410.147] | cls=Person | conf=0.289
2025-09-08T19:57:25.556Z | test-topic | sensor-4 | id=4 | bbox=[1283.630,264.106,1445.440,512.889] | cls=Person | conf=0.677
2025-09-08T19:57:25.596Z | test-topic | sensor-2 | id=1 | bbox=[12.460,742.930,172.411,1063.560] | cls=Person | conf=0.384
2025-09-08T19:57:26.756Z | test-topic | sensor-4 | id=4 | bbox=[1280.610,263.107,1450.950,513.143] | cls=Person | conf=0.

In [ ]:
#